In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import tensorflow as tf

# Load dataset
FILE_PATH = './Dataset/EURUSD/EURUSD_M30_features+label_v.2.1.csv'
data = pd.read_csv(FILE_PATH)

# Extract necessary columns
columns = ['Close', 'SMA200', 'SMA50', 'RSI14', 'signal']
data = data[columns]

# Handle label values and normalize features
data['signal'] = data['signal'].apply(lambda x: x if x in [1, 2, 3] else 3)
features = data[['Close', 'SMA200', 'SMA50', 'RSI14']].round(4)
labels = data['signal']

scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(features)

# Prepare sequences for transformer
seq_length = 15
X, y = [], []
for i in range(len(features_normalized) - seq_length):
    X.append(features_normalized[i:i + seq_length])
    y.append(labels.iloc[i + seq_length])

X = np.array(X)
y = np.array(y) - 1  # Adjust labels to be 0, 1, 2 for categorical encoding

# Build Transformer model
def transformer_encoder(inputs, num_heads, dropout_rate):
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
    attention_output = Dropout(dropout_rate)(attention_output)
    attention_output = Add()([inputs, attention_output])
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    
    ffn = Dense(inputs.shape[-1] * 4, activation="relu")(attention_output)
    ffn = Dense(inputs.shape[-1])(ffn)
    ffn_output = Dropout(dropout_rate)(ffn)
    ffn_output = Add()([attention_output, ffn_output])
    ffn_output = LayerNormalization(epsilon=1e-6)(ffn_output)
    
    return ffn_output

input_layer = Input(shape=(seq_length, features_normalized.shape[1]))
x = input_layer

for _ in range(2):
    x = transformer_encoder(x, num_heads=4, dropout_rate=0.4)

x = Dense(64, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(3, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=x)
model.compile(optimizer=Adam(learning_rate=0.005), loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Train the model
batch_size = 1024
epochs = 50

# Enable GPU
tf.config.set_visible_devices(tf.config.list_physical_devices('GPU'), 'GPU')

history = model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.2, verbose=1)

model.summary()


Epoch 1/50


ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(None,), output.shape=(None, 15, 3)